In [6]:
import json
import os
from dotenv import load_dotenv
from langchain.llms import AzureOpenAI
import openai
from IPython.display import display, HTML, JSON, Markdown

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

OPENAI_DAVINCI_DEPLOYMENT_NAME = os.getenv("OPENAI_DAVINCI_DEPLOYMENT_NAME")
OPENAI_DAVINCI_MODEL_NAME = os.getenv("OPENAI_DAVINCI_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

In [3]:
def init_llm(model=OPENAI_MODEL_NAME,
             deployment_name=OPENAI_DEPLOYMENT_NAME,
             openai_api_version=OPENAI_DEPLOYMENT_VERSION,
             temperature=0,
             max_tokens=400,
             top_p=1,
             ):

    llm = AzureOpenAI(deployment_name=deployment_name,
                      model=model,
                      openai_api_version=openai_api_version,
                      temperature=temperature,
                      max_tokens=max_tokens,
                      top_p=top_p,
                      model_kwargs={"stop": ["<|im_end|>"]}
                      )
    return llm

##### Let's do some math

In [4]:
# openAI gets this wrong!!
prompt = f""" Determine if the student's solution delimited by the triple backticks is correct or not.
Question: When I was 2 years old my sister was twice my age. I'm now 40 years old how old is my sister now? 
Student's answer: ```The sister is now 80 years old.``` 
If student is correct, then write 'Correct', otherwise write 'Incorrect'.  
<|im_end|>
"""

llm = init_llm()
sum = llm(prompt)
display(HTML(sum))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='aidemos-workshop.openai.azure.com', port=443): Max retries exceeded with url: //openai/deployments/gpt-35-turbo/completions?api-version=2023-03-15-preview (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000027453611330>: Failed to resolve 'aidemos-workshop.openai.azure.com' ([Errno 11001] getaddrinfo failed)")).
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: HTTPSConnectionPool(host='aidemos-workshop.openai.azure.com', port=443): Max retries exceeded with url: //openai/deployments/gpt-35-turbo/completions?api-version=2023-03-15-preview (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000027453612C20>: Failed to resol

##### Ask the model to solve it and then to compare both solutions and conclude which is correct.
 Define the task as a list of sub-tasks and ask the model to perform them in a specific order (splitting the task into sub-tasks technique).   
 Then ask the model to compare its own solution with the solution provided by the model and conclude which one is correct.
 Ask the model to share its reasoning for the conclusion.

In [5]:
Question = f"""When I was 2 years old my sister was twice of my age . I'm now 40 years old, how old is my sister now?"""
Student_Solution = f""" The sister is now 80 years old."""

prompt = f""" Determine if the Student's Solution for the Question is correct or not.
To solve the problem do the following:
1 - First, work out your OWN solution to the problem. Evaluate your final result to make sure it is correct and adheres to the question's conditions. 
    Reason about every step of your solution and make sure it is correct. 
2 - Second, compare your solution to the student's solution and evaluate if the student's solution is correct or not.
Don't decide if the student's solution is correct until you have done the problem yourself.

Student's Solution: ```{Student_Solution}```
Question: ```{Question}```


Use the following output format:
Actual solution steps: <your own solution steps>
\nStudent's solution: <student's solution>
\nStudent's solution is correct: <true/false>
<|im_end|>

"""

llm = init_llm()
sum = llm(prompt)
display(Markdown(sum))

Actual solution steps: 
- When I was 2 years old, my sister was 4 years old (2 * 2 = 4)
- The age difference between us is 2 years (4 - 2 = 2)
- Therefore, my sister is 42 years old now (40 + 2 = 42)

Student's solution: The sister is now 80 years old.

Student's solution is correct: False